In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-21 08:10:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2024-08-21 08:10:38 (15.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [4]:
print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [5]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
chars = sorted(list(set(text)))
vocab_sz = len(chars)
print(''.join(chars))
print(vocab_sz)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [24]:
#creating a simple tokenizer
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i: ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda c: ''.join([itos[i] for i in c])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
#encoding entirety of input text with tokenizer
import torch
data = torch.tensor(encode(text), dtype= torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
#90/10 data split
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

print(train[:10])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [10]:
#visualization of the autoregressive techniuque used by transformers for language modeling
block_sz = 8
x = train[:block_sz]
y = train[1:block_sz + 1]
for t in range(block_sz):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")


when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
torch.manual_seed(80)
batch_sz = 4 #number of input sequences to be processed in parallel
block_sz = 8 #maximum context length of predictions

def get_batch(split):
  #generate a small batch of data of inputs x and targets y
  data = train if split == 'train' else val
  ix = torch.randint(len(data) - block_sz, (batch_sz,))
  x = torch.stack([data[i:i+block_sz] for i in ix])
  y = torch.stack([data[i+1:i+block_sz+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_sz): #batch dimension
  for t in range(block_sz): #time dimension
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when the input is {context.tolist()}, the output is {target}")

inputs:
torch.Size([4, 8])
tensor([[52, 42,  1, 42, 47, 42,  1, 58],
        [39, 42, 50, 63,  1, 57, 54, 53],
        [51,  1, 58, 53,  1, 39, 50, 50],
        [63, 53, 59,  6,  1, 59, 52, 41]])
targets:
torch.Size([4, 8])
tensor([[42,  1, 42, 47, 42,  1, 58, 56],
        [42, 50, 63,  1, 57, 54, 53, 49],
        [ 1, 58, 53,  1, 39, 50, 50, 11],
        [53, 59,  6,  1, 59, 52, 41, 50]])
-----
when the input is [52], the output is 42
when the input is [52, 42], the output is 1
when the input is [52, 42, 1], the output is 42
when the input is [52, 42, 1, 42], the output is 47
when the input is [52, 42, 1, 42, 47], the output is 42
when the input is [52, 42, 1, 42, 47, 42], the output is 1
when the input is [52, 42, 1, 42, 47, 42, 1], the output is 58
when the input is [52, 42, 1, 42, 47, 42, 1, 58], the output is 56
when the input is [39], the output is 42
when the input is [39, 42], the output is 50
when the input is [39, 42, 50], the output is 63
when the input is [39, 42, 50, 63], 

### FEEDING OUR PREPROCESSED TRAINING DATA INTO NEURAL NETWORKS

1. THE BI-GRAM LANGUAGE MODEL
    A simple language model that predicts the next token in the sequence based on the single previous token. Can be extended to predict based on n preceding tokens (n-gram model)

In [17]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(80)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_sz):
      super().__init__()
      #each token directly reads off the logits for the next token in a lookup table
      self.token_embedding_table = nn.Embedding(vocab_sz, vocab_sz)

    def forward(self, idx, targets=None):
      #both idx and tensor are 2-dimensional (B,T) tensor of integers
      logits = self.token_embedding_table(idx) #(B, T, C)

      if targets == None:
        loss = None
      else:
        #since pytorch expects a (B,C,T) tensor, we reshape the tensor using torch.view()
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T) #combining our batch and time dimensions

        loss = F.cross_entropy(logits, targets) #negative log-likehood loss fn
      return logits, loss

    def generate(self, idx, max_new_tokens):
      for _ in range(max_new_tokens):
        # we pass our indexes to the forward fn to get our predictions(logits)
        logits , loss = self(idx)
        # focus on only the last time step
        logits = logits[:, -1, :] # becomes (B,C)
        #convert predictions to class probabilities
        probs = F.softmax(logits, dim=-1) #(B,C)
        # sample from the prob. distribution
        idx_next = torch.multinomial(probs, num_samples=1)
        # append sampled index to running sequence
        idx = torch.cat((idx, idx_next), dim=1)
      return idx


m = BigramLanguageModel(vocab_sz)
logits, loss = m(xb, yb)

print(logits)
print(logits.shape)
print(loss) #NOTE: ideal nll loss = -ln(1/vocab_sz)

tensor([[ 1.2811, -0.2446,  0.3116,  ...,  0.2522, -0.0340, -0.8544],
        [-0.0927, -1.2656, -2.1652,  ..., -0.0186,  0.9552,  1.3387],
        [-0.3464, -1.1768, -2.6356,  ...,  1.7470,  1.1502, -0.0674],
        ...,
        [ 1.0463, -0.3701,  1.5730,  ...,  0.1144,  1.5161,  1.6939],
        [ 1.2811, -0.2446,  0.3116,  ...,  0.2522, -0.0340, -0.8544],
        [ 0.9166, -0.5317, -0.2654,  ...,  0.3093, -0.0615, -0.7909]],
       grad_fn=<ViewBackward0>)
torch.Size([32, 65])
tensor(4.7332, grad_fn=<NllLossBackward0>)


In [25]:
char_seq = m.generate(torch.zeros((1,1), dtype= torch.long), max_new_tokens=100)[0].tolist() #generate a sequence of 100 characters using a newline character as initial idx
print(decode(char_seq)) #prev op returns vaiid tokenizer idxs which we then decode


trlkNuJab BzxGdt,NgmnMN?EnoLqX&J:,frIWyz$z3:kbxZ;syPJlDAvcGnwkGZKOIwwd;3!gfyzq;cK!Vk
mmG.!!Ia
TTNda'


In [26]:
#create our optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [27]:
batch_sz = 32
for steps in range(10000):
  #get a batch of data
  xb, yb = get_batch("train")

  #run through training loop
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

#print loss at tne end of training epochs
print(loss.item())

2.3386425971984863


In [28]:
#outputs after training
char_seq = m.generate(torch.zeros((1,1), dtype= torch.long), max_new_tokens=100)[0].tolist() #generate a sequence of 100 characters using a newline character as initial idx
print(decode(char_seq)) #prev op returns vaiid tokenizer idxs which we then decode


INRGLIfitsamy ONGonthow wolld.
Whours
dsensse, thilisich his thehur ofar ANUMourouictl h g loorepewh
